In [8]:
import pandas as pd
import tensorflow as tf
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys
from tabulate import tabulate

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas,today_log,recent_log

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    
    if filter_str == 'today':
        config_files=today_log(config_files)
    elif filter_str == 'recent':
        config_files=recent_log(config_files)
    elif filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            
            ed['dir']=cfg
            log_time=''
            for s in cfg.split(os.sep):
                if s.find('___')>=0:
                    log_time=s
                    break
            ed['log_time']=log_time
            for key,value in metrics.items():
                ed[key]=value
             
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

def dump(tags=['train/iou','val/iou'],
        rootpath=os.path.expanduser('~/tmp/logs/pytorch'),
        notes=['today','recent'],
        note_gtags=[['log_time'],
                    ['log_time']],
        descripe=['n_epoch'],
        delete_nan=False,
        dump_group=False,
        dump_dir=True):
    dir_tags=[tags[1],'dir']
    for note,ntag in zip(notes,note_gtags):
        show_tags=[]
        show_tags=tags.copy()
        group_tags=['note']+ntag
        show_tags+=group_tags
        show_tags+=descripe
        tasks=summary(rootpath,tags,note)
        if tasks.empty:
            print(note,ntag,'task is empty')
            continue

        # remove empty dirs
        if delete_nan:
            dirs=tasks[tasks[tags[1]].isna()]['dir'].tolist()
            tasks=tasks[tasks[tags[1]].notna()]
            for dir in dirs:
                print(os.path.dirname(dir))
                os.system('rm -rf {}'.format(os.path.dirname(dir)))

        #print(tasks[show_tags].groupby(group_tags).max().to_string())
        if dump_group:
            print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')
            print(tabulate(tasks[[tags[1]]+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
            print('\n')
        print(tabulate(tasks[show_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
        print('\n')
        if note=='recent':
            print(tabulate(tasks[show_tags].sort_values('log_time'),tablefmt='pipe',headers='keys'))
            print('\n')
        if dump_dir:
            print(tabulate(tasks[dir_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')
            


['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [9]:
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']
notes=['today','recent']
note_gtags=[['log_time'],
           ['log_time']]
dump(tags,rootpath,notes,note_gtags,dump_dir=True)

|    |   train/iou |   val/iou | note           | log_time              |   n_epoch |
|---:|------------:|----------:|:---------------|:----------------------|----------:|
|  3 |    0.132616 |  0.145005 | center_loss    | 2019-03-15___14-14-33 |       150 |
|  1 |    0.149774 |  0.172003 | wo_sum_feature | 2019-03-15___16-25-36 |       150 |
|  2 |    0.393983 |  0.362395 | center_loss    | 2019-03-15___15-19-56 |       150 |
|  0 |    0.565192 |  0.507013 | wo_center_loss | 2019-03-15___15-21-31 |       150 |


|    |   val/iou | dir                                                                                           |
|---:|----------:|:----------------------------------------------------------------------------------------------|
|  3 |  0.145005 | /home/yzbx/tmp/logs/pytorch/pspnet/cityscapes/center_loss/2019-03-15___14-14-33/config.txt    |
|  1 |  0.172003 | /home/yzbx/tmp/logs/pytorch/pspnet/cityscapes/wo_sum_feature/2019-03-15___16-25-36/config.txt |
|  2 |  0.362395 | /ho